In [ ]:
from __future__ import division, print_function
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Image, display, clear_output
import numpy as np
import matplotlib.pyplot as plt
import sklearn.datasets
import tensorflow as tf
import math
from tensorflow.python.framework.ops import reset_default_graph

from sklearn.utils import shuffle

# To speed up training we'll only work on a subset of the data containing only the numbers 0, 1.
# We discretize the data to 0 and 1 in order to use it with a 
# bernoulli observation model p(x|z) = Ber(mu(z))

def bernoullisample(x):
    return np.random.binomial(1,x,size=x.shape).astype('float32')

# Load data from compressed file with mnist
data = np.load('notebooks/lab5_Unsupervised/mnist.npz')

# Possible classes
classes = list(range(10))

# Set the classes we want to use.
included_classes = [0, 1, 4, 9] 

idxs_train = []
idxs_valid = []
idxs_test = []
num_classes = 0
for c in included_classes:
    if c in classes:
        num_classes += 1
        idxs_train += np.where(data['y_train'] == c)[0].tolist()
        idxs_valid += np.where(data['y_valid'] == c)[0].tolist()
        idxs_test += np.where(data['y_test'] == c)[0].tolist()

print("Number of classes included:", num_classes)
x_train = bernoullisample(data['X_train'][idxs_train]).astype('float32')
# Since this is unsupervised, the targets are only used for validation.
targets_train = data['y_train'][idxs_train].astype('int32')
x_train, targets_train = shuffle(x_train, targets_train, random_state=1234)

x_valid = bernoullisample(data['X_valid'][idxs_valid]).astype('float32')
targets_valid = data['y_valid'][idxs_valid].astype('int32')

x_test = bernoullisample(data['X_test'][idxs_test]).astype('float32')
targets_test = data['y_test'][idxs_test].astype('int32')

is_training_pl = tf.placeholder(tf.bool, name="is-training_pl")

print("training set dim(%i, %i)." % x_train.shape)
print("validation set dim(%i, %i)." % x_valid.shape)
print("test set dim(%i, %i)." % x_test.shape)

from tensorflow import layers
from tensorflow.contrib.layers import fully_connected, convolution2d, convolution2d_transpose, batch_norm, max_pool2d, dropout
from tensorflow.python.ops.nn import relu, elu, relu6, sigmoid, tanh, softmax, softplus

# Error functions    
def sum_of_squared_errors(p, t):
    return tf.reduce_sum(tf.square(p - t), axis=[1])


# computing cross entropy per sample
def categorical_cross_entropy(p, t, eps=1e-10):
    return -tf.reduce_sum(t * tf.log(p+eps), axis=[1])


def binary_cross_entropy(p, t, eps=1e-10):
    return -tf.reduce_sum(t * tf.log(p+eps) + (1-t) * tf.log(1-p+eps), axis=-1)


def kl_normal2_stdnormal(mean, log_var, eps=0.0):
    return -0.5 * tf.reduce_sum(1 + log_var - tf.square(mean) - tf.exp(log_var), axis=1)


c = - 0.5 * math.log(2*math.pi)
def log_normal2(x, mean, log_var, eps=0.0):
    return tf.reduce_sum(c - log_var/2 - tf.square(x - mean) / (2 * tf.exp(log_var) + eps), axis=[1])


# reset graph
reset_default_graph()

# -- THE MODEL --#
num_channels = 1; #Black and white for MNIST
k = 16;

# Layer definitions
def layer(x, units):
    x = batch_norm(x, scope='layer_batch_norm')
    x = fully_connected(x, num_outputs=units, activation_fn=relu,
                     normalizer_fn=batch_norm, scope="layer_ReLU")
    x = convolution2d(x, num_outputs=units, kernel_size=(3, 3),
                         stride=1, scope="layer_convolution")
    return dropout(x, is_training=is_training_pl, scope="layer_dropout")
    
def denseBlock(x, n_layers):
    res = []
    for i in range(n_layers):
        layer_output = layer(x, k)
        x = tf.concat(x, layer_output)
        res = tf.concat(res, layer_output)
    return res
    

def transitionUp(x, units):
    x = convolution2d_transpose(x, num_outputs=units, kernel_size=(3, 3), stride=2)
    
    
def transitionDown(x, units, pooling=True):
    x = batch_norm(x, scope='tdown_batch_norm')
    x = fully_connected(x, num_outputs=units, activation_fn=relu,
                     normalizer_fn=batch_norm, scope="tdown_ReLU")
    x = convolution2d(x, num_outputs=units, kernel_size=(1, 1),
                         stride=1, scope="tdown_convolution")
    x = dropout(x, is_training=is_training_pl, scope="tdown_dropout")
    if pooling:
        x = max_pool2d(x, kernel_size=(2, 2), scope="tdown_max_pooling")
    return x
    


# Input placeholder
x_pl = tf.placeholder(tf.float32, [None, num_channels], 'x_pl')
print('x_pl', x_pl.shape)

# DOWN SAMPLING
x = convolution2d(x_pl, num_outputs=k, kernel_size=(3, 3),
                         stride=1, scope="pre-convolution")
    
skip1 = denseBlock(x, 4)
x = tf.concat(x, skip1)
x = transitionDown(x, 64, True)

skip2 = denseBlock(x, 4)
x = tf.concat(x, skip2)
x = transitionDown(x, )

# BOTTLENECK
denseBlock(x, )

# UPSAMPLING
